# Recomendador de artigos - Open Alex e GPT

In [70]:
import requests
import pandas as pd
import os
import pyarrow.parquet as pq


pd.options.display.max_columns = 999

In [71]:
def coletar_dados_e_salvar(data_inicial,data_fim):
    
    cursor = '*'
            
    contador = 1
    contador_erro = 0
    
    diretorio = f'datasets_{data_inicial}_to_{data_fim}'
    
    if not os.path.exists(diretorio):
        os.makedirs(diretorio)
    
    while cursor != None:
        url = f'https://api.openalex.org/works?filter=from_publication_date:{data_inicial},to_publication_date:{data_fim},type:journal-articlee&per-page=200&cursor={cursor}'

        
        try:
            requisicao = requests.get(url)
            pagina_com_resultados = requisicao.json()
            
        except (requests.exceptions.RequestException, ValueError) as e:
            contador_erro += 1 
            print(f"Erro na página {contador} (Erro {contador_erro}): {e}")
            continue 
        
        resultados = pagina_com_resultados.get('results', [])
        
        df = pd.DataFrame(resultados)
        parquet_arquivo = os.path.join(diretorio, f'registros_{data_inicial}_to_{data_fim}_{contador}.parquet')
        df.to_parquet(parquet_arquivo, index=False)
        
        cursor = pagina_com_resultados['meta'].get('next_cursor')
        
        contador += 1
    
    print(f"Total de erros: {contador_erro}")

In [72]:
def concatenar_arquivos_parquet(folder_path):

    dataframes = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.parquet'):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_parquet(file_path)
            dataframes.append(df)

    df_concatenado = pd.concat(dataframes, ignore_index=True)
    
    df_concatenado.to_parquet('df_concatenado.parquet',index=False)
    

## CONTINUAR

In [87]:
def extrair_concepts_scores(df):
    concept_data = []

    for concepts in df['concepts']:
        concept_scores = {}
        for concept in concepts:
            if concept['level'] < 2:
                concept_scores[concept['display_name']] = concept['score']
        concept_data.append(concept_scores)

    df_concepts = pd.DataFrame(concept_data, index=df.index).fillna(0).round(4)

    df_final = pd.concat([df, df_concepts], axis=1)

    return df_final

In [74]:
def get_source_name(primary_location):
    if primary_location is None:
        return None
    source_info = primary_location.get('source', None)
    
    if source_info is None:
        return None
    display_name = source_info.get('display_name', None)
    
    return display_name



def transformar_df(df):
    
    df = df.loc[:,['doi','title','publication_date','primary_location','open_access','concepts']]
    
    df_filtrado = df.copy()
    
    df_filtrado.loc[:,'source_title'] = df_filtrado['primary_location'].apply(get_source_name)
    
    df_filtrado.loc[:,'open_access_status'] = df_filtrado['open_access'].apply(lambda x: x.get('is_oa',None))
    
    df_filtrado = extrair_concepts_scores(df_filtrado)
    
    df_filtrado.drop(columns=['primary_location','open_access','concepts'],inplace=True)
    
    return df_filtrado

In [75]:
def ler_e_transformar_arquivos_diretorio(diretorio_input, diretorio_output):
    
    if not os.path.exists(diretorio_output):
        os.makedirs(diretorio_output)
    
    arquivos = [arquivo for arquivo in os.listdir(diretorio_input) if arquivo.endswith(".parquet")]
    
    try:
        for arquivo_parquet in arquivos:
            caminho_input = os.path.join(diretorio_input, arquivo_parquet)
            df_original = pd.read_parquet(caminho_input)  
            df_processado = transformar_df(df_original) 
            
            output_file = os.path.splitext(arquivo_parquet)[0] + ".parquet"
            caminho_output = os.path.join(diretorio_output, output_file)
            
            df_processado.to_parquet(caminho_output, index=False)

    except Exception as error:
        print(arquivo_parquet,error)

In [76]:
def concatenar_arquivos_parquet(diretorio_input):
    dfs = []

    for arquivo in os.listdir(diretorio_input):
        if arquivo.endswith('.parquet'):
            file_path = os.path.join(diretorio_input, arquivo)
            df_parquet = pd.read_parquet(file_path)
        
            dfs.append(df_parquet)

    df_concatenado = pd.concat(dfs, ignore_index=True)
    
    df_concatenado.iloc[:,7:] = df_concatenado.iloc[:,7:].fillna(0)
    

    return df_concatenado

In [77]:
def atribuir_concepts(df):
    concepts = df.columns[7:]
    
    concept_lists = [list(concepts[row > 0]) for _, row in df[concepts].iterrows()]


    df_concepts = pd.DataFrame({'concepts': [';'.join(concept_list) for concept_list in concept_lists]})

    df = pd.concat([df, df_concepts], axis=1)

    
    
    return df

In [68]:
data_inicial = '2023-09-29'
data_final = '2023-09-29'

coletar_dados_e_salvar(data_inicial,data_final)

Total de erros: 0


In [69]:
diretorio_input = 'datasets_2023-09-29_to_2023-09-29'
diretorio_output = 'datasets_2023-09-29_to_2023-09-29_processados'

In [88]:
ler_e_transformar_arquivos_diretorio(diretorio_input,diretorio_output)

In [89]:
df_concatenado = concatenar_arquivos_parquet(diretorio_output)

In [90]:
df_final = atribuir_concepts(df_concatenado)

In [91]:
df_final.shape

(11068, 308)

In [92]:
df_final.to_parquet('df_concatenado.parquet')

# ------------RODAR NOVAMENTE----------

___

In [93]:
df = pd.read_parquet('df_concatenado.parquet')

In [94]:
df

,doi,title,publication_date,source_title,open_access_status,Chemical engineering,Nuclear chemistry,Materials science,Chemistry,Nanotechnology,Organic chemistry,Composite material,Fishery,Engineering,Biology,Medicine,Anesthesia,Mechanical engineering,Pathology,Environmental health,Psychiatry,Internal medicine,Surgery,Endocrinology,Gastroenterology,Virology,Paleontology,Computer science,Artificial intelligence,Data mining,Machine learning,Mathematics,Statistics,Political science,Law,Humanities,Philosophy,Geotechnical engineering,Reliability engineering,Structural engineering,Geography,Physics,Cartography,Quantum mechanics,World Wide Web,Cell biology,Molecular biology,Immunology,Biochemistry,Psychology,Nursing,Sociology,Social psychology,Social science,Economy,Economics,Cognitive psychology,Developmental psychology,Audiology,Neuroscience,Management,Psychotherapist,Medicinal chemistry,Computational chemistry,Photochemistry,Botany,Horticulture,Medical education,Public relations,Pedagogy,Business,Physical therapy,Marketing,Microeconomics,Pulp and paper industry,Physical chemistry,Information retrieval,Finance,Financial economics,Medical physics,Astrophysics,Archaeology,Art,Geology,Combinatorial chemistry,Macroeconomics,Family medicine,Operations management,Geometry,Programming language,Data science,Pure mathematics,Epistemology,Oncology,Astrobiology,Algorithm,Mathematical analysis,Database,Microbiology,Genetics,Anatomy,Evolutionary biology,Environmental science,Petroleum engineering,Oceanography,Linguistics,Optics,Metallurgy,Thermodynamics,Gender studies,Economic growth,Optoelectronics,Demography,Cancer research,Bioinformatics,Physical medicine and rehabilitation,Pediatrics,Combinatorics,Socioeconomics,Agricultural economics,Demographic economics,History,Ancient history,Classics,Literature,Aesthetics,Condensed matter physics,Classical mechanics,Atomic physics,Stereochemistry,Crystallography,Computational biology,Clinical psychology,Electrical engineering,Agronomy,Obstetrics,Human–computer interaction,Telecommunications,Chromatography,Radiology,Ophthalmology,Gynecology,Applied psychology,Operating system,Meteorology,Climatology,Gerontology,Advertising,Anthropology,Econometrics,Control engineering,Applied mathematics,Aerospace engineering,Water resource management,Environmental engineering,Ecology,Waste management,Food science,Traditional medicine,Visual arts,Criminology,Medical emergency,Astronomy,Engineering physics,Public administration,Mathematics education,Psychoanalysis,Cognitive science,Nuclear physics,Operations research,Library science,Engineering ethics,Multimedia,Engineering management,Statistical physics,Architectural engineering,Financial system,Monetary economics,Quantum electrodynamics,Particle physics,Theoretical physics,Mathematical physics,Physiology,Mathematical optimization,Distributed computing,Real-time computing,Computer network,Economic system,Dentistry,Economic geography,Industrial organization,Remote sensing,Environmental ethics,Art history,Theology,Political economy,Acoustics,Mineralogy,Soil science,Biotechnology,Biochemical engineering,Process engineering,Mechanics,Embedded system,Computer architecture,Software engineering,Computer security,Systems engineering,Animal science,Atmospheric sciences,Intensive care medicine,Chemical physics,Pharmacology,Seismology,Parallel computing,Computer engineering,Computer hardware,General surgery,Geochemistry,Orthodontics,Law and economics,Inorganic chemistry,Transport engineering,Process management,Risk analysis (engineering),Environmental economics,Nuclear magnetic resonance,Geophysics,Petrology,Physical geography,Arithmetic,Speech recognition,Communication,Biomedical engineering,Polymer chemistry,Biophysics,Veterinary medicine,Business administration,Computer vision,Religious studies,International trade,Development economics,Internet privacy,Environmental planning,Biological system,Electronic engineering,Geodesy,Simulation,Knowledge management,Andrology,Marine engineer

In [95]:
df.concepts.value_counts(normalize=True)

concepts
Computer science                                                                                                           0.016082
Political science;Humanities;Philosophy                                                                                    0.006596
Humanities;Philosophy                                                                                                      0.005511
Humanities;Art                                                                                                             0.005150
Medicine;Internal medicine;Surgery                                                                                         0.005060
                                                                                                                             ...   
Humanities;Physics;Psychology;Art                                                                                          0.000090
Mathematics;Humanities;Art                                         

In [96]:
for i in df.iloc[:,7:].columns:
    print(i)

Materials science
Chemistry
Nanotechnology
Organic chemistry
Composite material
Fishery
Engineering
Biology
Medicine
Anesthesia
Mechanical engineering
Pathology
Environmental health
Psychiatry
Internal medicine
Surgery
Endocrinology
Gastroenterology
Virology
Paleontology
Computer science
Artificial intelligence
Data mining
Machine learning
Mathematics
Statistics
Political science
Law
Humanities
Philosophy
Geotechnical engineering
Reliability engineering
Structural engineering
Geography
Physics
Cartography
Quantum mechanics
World Wide Web
Cell biology
Molecular biology
Immunology
Biochemistry
Psychology
Nursing
Sociology
Social psychology
Social science
Economy
Economics
Cognitive psychology
Developmental psychology
Audiology
Neuroscience
Management
Psychotherapist
Medicinal chemistry
Computational chemistry
Photochemistry
Botany
Horticulture
Medical education
Public relations
Pedagogy
Business
Physical therapy
Marketing
Microeconomics
Pulp and paper industry
Physical chemistry
Informat

In [97]:
escolhas = ['Library science']


filtro = df[escolhas].gt(0).all(axis=1)  
df_filtrado = df[filtro]

colunas_desejadas = ['doi', 'title', 'publication_date', 'open_access_status','concepts'] + escolhas
df_colunas_filtrado = df_filtrado[colunas_desejadas]

df_colunas_filtrado['prob_sum'] = df_colunas_filtrado.loc[:,escolhas].sum(axis=1)

df_colunas_filtrado.sort_values(by='prob_sum',ascending=False).head(20)


/tmp/ipykernel_3880/771015353.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_colunas_filtrado['prob_sum'] = df_colunas_filtrado.loc[:,escolhas].sum(axis=1)


,doi,title,publication_date,open_access_status,concepts,Library science,prob_sum
8827,https://doi.org/10.33077/uw.25448730.zbkh.2023...,Some Remarks about the Cyrillo-Methodian Bibli...,2023-09-29,True,Computer science;Political science;World Wide ...,0.6547,0.6547
7097,https://doi.org/10.1002/nadc.20234136126,Notizen aus der Chemie,2023-09-29,False,Computer science;Art;Library science;Art history,0.5924,0.5924
1567,https://doi.org/10.1111/aman.13933,On deadlines,2023-09-29,False,Medicine;Computer science;World Wide Web;Histo...,0.5807,0.5807
10149,https://doi.org/10.1111/opo.13237,"Optics of the human eye, Second EditionBy Davi...",2023-09-29,True,Computer science;Algorithm;History;Library sci...,0.5709,0.5709
246,https://doi.org/10.5070/ac3.1680,"Cosmovisiones/Cosmovisões, 1(1) Abstracts",2023-09-29,True,Computer science;Political science;Law;Philoso...,0.5681,0.5681
7742,https://doi.org/10.1111/puar.13729,"Maher, Craig, Sungho Park, Bruce D. McDonald I...",2023-09-29,False,Computer science;Political science;Sociology;L...,0.5620,0.5620
2175,https://doi.org/10.1093/yiel/yvad048,5. United States-Mexico-Canada Agreement (USMC...,2023-09-29,False,Political science;Law;History;Library science;...,0.5611,0.5611
10516,https://doi.org/10.1021/acsmedchemlett.3c00412,Empowering Women in Organic Chemistry (EWOC) a...,2023-09-29,False,Chemistry;Computer science;Library science,0.5336,0.5336
10580,https://doi.org/10.1021/acsorginorgau.3c00050,Empowering Women in Organic Chemistry (EWOC) a...,2023-09-29,True,Computer science;Political science;Law;History...,0.5271,0.5271
2932,https://doi.org/10.1002/nop2.2009,Research trends in readiness for hospital disc...,2023-09-29,True,Medicine;Pathology;Computer science;Data scien...,0.5133,0.5133


In [8]:
df.shape

(72986, 149)

In [9]:
df_teste = df.query('`Computer science` > 0').query('Medicine > 0').query('Psychology > 0')
df_teste.shape

(956, 149)

In [6]:
escolhas = ['Computer science','Medicine','Psychology']

df_teste = df.query('`Computer science` > 0').query('Medicine > 0').query('Psychology > 0')
df_teste['prob_sum'] = df_teste[['Computer science','Medicine','Psychology']].sum(axis=1)

df_teste.sort_values(by='prob_sum',ascending=False)

,doi,title,publication_date,created_date,cited_by_count,source_title,open_access_status,Medicine,Psychology,Political science,Computer science,Economics,Physics,Engineering,Chemistry,Mathematics,Business,Biology,Sociology,Philosophy,Environmental science,Geography,Geology,Materials science,Art,Cardiology,Combinatorics,Accounting,Humanities,History,Condensed matter physics,Internal medicine,Epistemology,Chromatography,Microbiology,Optics,Structural engineering,Virology,Seismology,Artificial intelligence,Astrophysics,Particle physics,Computer vision,Endocrinology,Composite material,Nanotechnology,Surgery,Ecology,Dentistry,Geotechnical engineering,Nuclear engineering,Pharmacology,Data mining,Geochemistry,Andrology,Mechanics,Polymer chemistry,Traditional medicine,Metallurgy,Food science,Transport engineering,Geodesy,Climatology,Genetics,Optoelectronics,Cell biology,Advertising,Computer security,Law,Immunology,Information retrieval,Mathematical optimization,Neuroscience,Petroleum engineering,Aesthetics,Natural language processing,Ophthalmology,Photochemistry,Linguistics,Anesthesia,Reliability engineering,Dermatology,Gynecology,Physical therapy,Anatomy,Biochemistry,Veterinary medicine,Theology,Mathematics education,Programming language,Fishery,Archaeology,Agronomy,Atomic physics,Audiology,Demography,Psychotherapist,Machine learning,Developmental psychology,Telecommunications,Paleontology,Botany,Economic geography,Environmental chemistry,Gastroenterology,Algorithm,Pediatrics,Marketing,Crystallography,Database,Obstetrics,Remote sensing,Oceanography,Biochemical engineering,Aerospace engineering,Cognitive psychology,Acoustics,Computer graphics (images),Cancer research,Psychiatry,Computer network,Econometrics,Pathology,Urology,Nuclear medicine,Internet privacy,Human–computer interaction,Atmospheric sciences,Animal science,Thermodynamics,Statistics,Radiochemistry,Actuarial science,Social psychology,Astrobiology,Agricultural science,Molecular biology,Pure mathematics,Industrial organization,Automotive engineering,Physical medicine and rehabilitation,Operating system,Radiology,Waste management,Optometry,Architectural engineering,Marine engineering,Mathematical economics,concepts,prob_sum
64808,https://doi.org/10.2196/44985,User-centered development of a mobile applicat...,2023-09-26,2023-08-25,0,JMIR cancer,True,0.3835,0.4387,0.0000,0.4391,0.0000,0.0000,0.2036,0.0,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Medicine;Psychology;Computer science;Engineering,1.2613
26469,https://doi.org/10.1108/jet-12-2022-0069,Real-time tactile biofeedback device use for i...,2023-09-29,2023-09-29,0,Journal of enabling technologies,False,0.4131,0.4120,0.0000,0.4334,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Medicine;Psychology;Computer science,1.2585
33205,https://doi.org/10.1093/jamia/ocad167,Deep sequential neural network models improve ...,2023-09-28,2023-09-29,0,Journal of the American Medical Informatics As...,True,0.4264,0.3708,0.0000,0.4353

In [42]:
df[df.concepts.str.contains('Computer science')][df.concepts.str.contains('Medicine')]['concepts']

/tmp/ipykernel_4585/3981352342.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df.concepts.str.contains('Computer science')][df.concepts.str.contains('Medicine')]['concepts']


2                                Medicine;Computer science
4         Medicine;Psychology;Computer science;Mathematics
6                                Medicine;Computer science
65          Medicine;Computer science;Engineering;Business
68                               Medicine;Computer science
                               ...                        
72940                            Medicine;Computer science
72957    Medicine;Psychology;Political science;Computer...
72963       Medicine;Psychology;Computer science;Geography
72964                 Medicine;Computer science;Philosophy
72985        Medicine;Psychology;Computer science;Business
Name: concepts, Length: 2774, dtype: object